In [3]:
import numpy as np
import json

In [4]:
import tensorflow as tf

In [5]:
json_filename = '/home/developer/gcp/cbidmltsf/parameters/EDSLSTM_TPU_011.json'

In [6]:
# get the model parameters from configuration json file
with open(json_filename, 'r') as input_file:
    model_params = json.load(input_file)

In [7]:
model_params['encoder']

{'no_hidden': [256, 256, 256],
 'activation': ['elu', 'elu', 'elu'],
 'dropout': [0.2, 0.2, 0.2],
 'recurrent_dropout': [0.2, 0.2, 0.2],
 'momentum_h': 0.6,
 'momentum_c': 0.6}

In [8]:
indexes = list(np.arange(len(model_params['encoder']['no_hidden'])))

In [11]:
no_hiddens = model_params['encoder']['no_hidden']
activations = model_params['encoder']['activation']
dropouts = model_params['encoder']['dropout']
recurrent_dropouts = model_params['encoder']['recurrent_dropout']

In [20]:
# a dict to store all layers of the encoder
encoder = dict()

In [21]:
for index, no_hidden, activation, dropout, recurrent_dropout in zip(indexes,
                                                                    no_hiddens,
                                                                    activations,
                                                                    dropouts,
                                                                    recurrent_dropouts):
    '''
    zip returns an iterable like this one
    0 256 elu 0.2 0.2
    1 256 elu 0.2 0.2
    2 256 elu 0.2 0.2
    '''
    
    level_key = 'level_{}'.format(index)
    encoder[level_key] = tf.keras.layers.LSTM(
        units=no_hidden,
        activation=activation,
        dropout=dropout,
        recurrent_dropout=recurrent_dropout,
        # LSTMs in all levels return sequences
        return_sequences=True,
        # LSTM in the uppermost level returns also the state
        return_state=index==indexes[-1],
        name='encoder_{}'.format(level_key))

In [26]:
# the encoder dictionary stores all the stacked LSTMs
encoder

{'level_0': <tensorflow.python.keras.layers.recurrent_v2.LSTM at 0x7fce1034ab90>,
 'level_1': <tensorflow.python.keras.layers.recurrent_v2.LSTM at 0x7fce11a98890>,
 'level_2': <tensorflow.python.keras.layers.recurrent_v2.LSTM at 0x7fce90082fd0>}

In [28]:
# now build a list to generalize the encoder output management
encoder_output = list()

In [ ]:
# initialize the encoder output list with the features
encoder_output.append(features['hourly'])

In [30]:
# build a list of level keys (on indexes) to iterate on
level_keys = ['level_{}'.format(index) for index in indexes]

In [31]:
level_keys

['level_0', 'level_1', 'level_2']

In [ ]:
# now process the encoder stack of LSTMs as a cascade:
# list[n] is the input for list[n+1]
for level_key in level_keys:
    encoder_output.append(encoder[level_key](encoder_output[-1]))
    
    '''
    encoder['level_0'] LSTM receives features['hourly'] and returns encoder_stack_h0 (return_sequences=True)
    encoder['level_1'] LSTM receives encoder_stack_h0 and returns encoder_stack_h1 (return_sequences=True)
    encoder['level_2'] LSTM receives encoder_stack_h1 and returns the tuple
                       (encoder_stack_h2, encoder_last_h2, encoder_last_c2)
                       (return_sequences=True, return_state=True)
    '''

In [8]:
model_params['decoder']

{'no_hidden': [256, 256],
 'activation': ['elu', 'elu'],
 'dropout': [0.2, 0.2],
 'recurrent_dropout': [0.2, 0.2]}

In [9]:
# generalize now the decoder object construction

In [10]:
# a list of indexes for the decoder layers
# use any decoder feature list to get the number of layers
indexes = list(np.arange(len(model_params['decoder']['no_hidden'])))

In [11]:
# load all the decoder feature lists
no_hiddens = model_params['decoder']['no_hidden']
activations = model_params['decoder']['activation']
dropouts = model_params['decoder']['dropout']
recurrent_dropouts = model_params['decoder']['recurrent_dropout']

In [14]:
# a dict to store all layers of the decoder
decoder = dict()

In [15]:
for index, no_hidden, activation, dropout, recurrent_dropout in zip(indexes,
                                                                    no_hiddens,
                                                                    activations,
                                                                    dropouts,
                                                                    recurrent_dropouts):
    '''
    zip returns an iterable like this one
    0 256 elu 0.2 0.2
    1 256 elu 0.2 0.2
    2 256 elu 0.2 0.2
    '''

    level_key = 'level_{}'.format(index)
    decoder[level_key] = tf.keras.layers.LSTM(
        units=no_hidden,
        activation=activation,
        dropout=dropout,
        recurrent_dropout=recurrent_dropout,
        # LSTMs in all levels return sequences
        return_sequences=True,
        # LSTMs in all levels do not return state
        return_state=False,
        name='decoder_{}'.format(level_key))

In [16]:
decoder

{'level_0': <tensorflow.python.keras.layers.recurrent_v2.LSTM at 0x7fbc863feed0>,
 'level_1': <tensorflow.python.keras.layers.recurrent_v2.LSTM at 0x7fbc864159d0>}

In [26]:
# now build a list to generalize the decoder output management
decoder_output = list()

In [ ]:
# initialize the decoder output list with the TimeDistributed decoder input
decoder_output.append(decoder_input)

In [27]:
# build a list of level keys (on indexes) to iterate on
level_keys = ['level_{}'.format(index) for index in indexes]

In [28]:
level_keys

['level_0', 'level_1']

In [4]:
level_key = 'level_1'

In [5]:
initial_state = [3, 3] if level_key == 'level_0' else None

In [6]:
initial_state

In [ ]:
# now process the decoder stack of LSTMs as a cascade:
# list[n] is the input for list[n+1]
for level_key in level_keys:
    # last hidden state and last cell state in encoder stack
    # are passed as initial state for first decoder layer
    initial_state = [encoder_last_hn, encoder_last_cn] if level_key == 'level_0' else None
    # ensure initial_state is passed as a call argument
    decoder_output.append(decoder[level_key](decoder_output[-1], initial_state=initial_state))